# Practical Exam: Bakehouse

Mayu's Bakehouse is a local bakery known for its diverse selection of high quality, artisanal baked goods.

The store wants to identify which of its baked goods are most popular with customers, and have the highest sales.

With this information, the store can adjust its focus to the most profitable baked goods, improving its overall business performance.

## Data

The dataset is available in the table `bakery_data`.

The dataset contains data on the last quarter's sales at Mayu's Bakehouse.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| product_id    | Nominal. The unique identifier of the product.<br>Missing values are not possible due to the database structure.  |
| category    | Categorical.  The category of the product, one of 6 values (Bread, Pastries, Cakes, Cookies, Muffins, Donuts). <br> Missing values should be replaced with "Donuts".|
| item_type    | Categorical. Indicates the bakery item type. Values: "Signature Item", "Seasonal", "Limited", and "Standard".  <br> Missing values should be replaced with "Standard". |
| dietary_options    | Categorical. Dietary labels for the bakery item. Values: "Conventional", "Gluten-free", "Vegan", "Organic". <br> Missing values should be replaced with "Conventional". |
| sweet_intensity    | Categorical. Categorisation of how much sugar the product contains. Values: "Mild/Subtle", "Moderate", "Intense" . <br> Missing values should be replaced with 'Mild/Subtle'.|
| price    | Continuous. The price of the bakery item. Any positive value, rounded to 2 decimal places.  <br> Missing values should be replaced with the default retail price of 5.|
| units_sold    | Discrete. The number of units sold of the item.  <br> Missing values should be replaced with the average of units sold. |
| average_rating    | Continuous. The average rating of the bakery item, expressed as a percentage given by customers. Ranges from 0 to 100.  <br> Missing values should be replaced with the most frequent value. |

# Task 1

Not all of the family are familiar with working with data and don't know how important it is to accurately record information. For standard items, not all family members enter the `item_type`. It is believed to be missing for some items. 

Write a query to calculate how many items have the `item_type` value missing. Your output should be a single column, `missing_item`, with a single row giving the number of missing values.

In [38]:
-- Write your query for task 1 in this cell
SELECT COUNT(item_type) AS missing_item
FROM bakery_data
WHERE item_type ='-';

,missing_item
0,105


# Task 2

Now that you know that not all of the family accurately record all of the data, you need to make sure the rest of the data matches what you are expecting before you start any analysis. 

The table below describes what the data should look like. 

Write a query to ensure the data matches the description provided. 

Do not update the original table. The DataFrame that you return should be called `clean_data`. 

<br>

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| product_id    | Nominal. The unique identifier of the product.<br>Missing values are not possible due to the database structure.  |
| category    | Categorical.  The category of the product, one of 6 values (Bread, Pastries, Cakes, Cookies, Muffins, Donuts) <br> Missing values should be replaced with "Donuts"|
| item_type    | Categorical. Indicates the bakery item type. Values: "Signature Item", "Seasonal", "Limited", and "Standard".  <br> Missing values should be replaced with "Standard" |
| dietary_options    | Categorical. Dietary labels for the bakery item. Values: "Conventional", "Gluten-free", "Vegan", "Organic" <br> Missing values should be replaced with "Conventional". |
| sweet_intensity    | Categorical. Categorisation of how much sugar the product contains. Values: "Mild/Subtle", "Moderate", "Intense"  <br> Missing values should be replaced with 'Mild/Subtle'.|
| price    | Continuous. The price of the bakery item. Any positive value, rounded to 2 decimal places.  <br> Missing values should be replaced with the default retail price of 5.|
| units_sold    | Discrete. The number of units sold of the item.  <br> Missing values should be replaced with the average of units sold. |
| average_rating    | Continuous. The average rating of the bakery item, expressed as a percentage given by customers. Ranges from 0 to 100.  <br> Missing values should be replaced with the most frequent value. |

In [39]:
-- Write your query for task 2 in this cell
SELECT product_id,
		category,
		CASE WHEN item_type = '-' THEN 'Standard' ELSE item_type END AS item_type,
		dietary_options,
		REPLACE(sweet_intensity,'Moderatrr', 'Moderate') AS sweet_intensity,
		COALESCE(ROUND(CAST(NULLIF(REGEXP_REPLACE(price, '[^\d.]', '', 'g'), '') AS DECIMAL(10, 2)), 2), 5.00) AS price,
		units_sold,
		average_rating		
FROM bakery_data;



,product_id,category,item_type,dietary_options,sweet_intensity,price,units_sold,average_rating
0,1000,Pastries,Signature Item,Organic,Moderate,6.08,64,57.07
1,1001,Cookies,Standard,Organic,Moderate,8.15,23,51.79
2,1002,Pastries,Standard,Vegan,Intense,8.16,52,81.72
3,1003,Donuts,Seasonal,Organic,Intense,5.58,33,48.64
4,1004,Bread,Signature Item,Vegan,Moderate,9.34,29,79.51
...,...,...,...,...,...,...,...,...
1495,2495,Pastries,Seasonal,Conventional,Moderate,8.10,44,92.29
1496,2496,Cookies,Standard,Vegan,Moderate,7.04,18,93.45
1497,2497,Muffins,Signature Item,Organic,Moderate,5.00,80,59.36
1498,2498,Muffins,Standard,Vegan,Intense,5.69,98,83.73


# Task 3

The bakery aims to create high quality products for all of its customers, even if they have special dietary needs. They want to confirm that products get similar ratings for all dietary options.  

Write a query to return the `dietary_options`, `median_avg_rating`, `min_avg_rating` and `max_avg_rating` columns. 

You should use the original `bakery_data` table for this task. The DataFrame that you return should be called `min_max_rating`. 

In [40]:
-- Write your query for task 3 in this cell
SELECT dietary_options,
		percentile_cont(0.5) within group (order by average_rating asc) as median_avg_rating,
		min(average_rating) as min_avg_rating,
		max(average_rating) as max_avg_rating
from bakery_data
group by dietary_options

,dietary_options,median_avg_rating,min_avg_rating,max_avg_rating
0,Conventional,89.86,68.16,100.00
1,Gluten-free,64.12,46.28,85.56
2,Organic,53.98,30.19,74.61
3,Vegan,80.51,55.09,99.63


# Task 4

The staff want to look in more detail at cake and muffin products where the number units sold was greater than the average number of units sold for cakes and muffins. 

Write a query to return the `product_id`, `units_sold` and `average_rating` of the rows of interest to the team. 

You should start with the original `bakery_data` table. The DataFrame that you return should be called `products_high_units`.

In [41]:
-- Write your query for task 4 in this cell
select product_id,units_sold, average_rating
from bakery_data
where units_sold > (
	select avg(units_sold)
	from bakery_data
	where category in ('Cakes', 'Muffins')) AND category in('Cakes', 'Muffins')


,product_id,units_sold,average_rating
0,1018,89,69.16
1,1020,100,94.21
2,1022,95,56.27
3,1035,102,98.28
4,1036,69,92.70
...,...,...,...
174,2480,75,90.31
175,2486,114,65.74
176,2489,108,95.10
177,2497,80,59.36
